In [26]:
import pandas as pd
import glob

In [27]:
# dados base
df_base = pd.concat([pd.read_excel(f) for f in glob.glob("extrato/*.xls*")], ignore_index=True)
df_base.columns = ["Data", "Tipo", "Historico", "Valor"]

In [28]:
# filtro base 
df_filtro = df_base[
    (df_base['Historico'] != 'HISTÓRICO') &
    (df_base['Historico'].notna())
   ]

In [29]:
# tratamento de linhas abaixo de historico
linhas_agrupadas = []
temp = {}

for _, row in df_filtro.iterrows():
    if pd.notna(row["Data"]): 
        if temp:
            linhas_agrupadas.append(temp)
        temp = {
            "Data": row["Data"],
            "Tipo": row["Tipo"],
            "Historico": row["Historico"],
            "Valor": row["Valor"],
            "SaldoDia": "",
            "Extras": []
        }
    else:
        hist = str(row["Historico"]) if pd.notna(row["Historico"]) else ""
        val = row["Valor"]
        if pd.notna(val):
            temp["SaldoDia"] = val
        elif hist:
            temp["Extras"].append(hist)

if temp:
    linhas_agrupadas.append(temp)

df_formatado = pd.DataFrame(linhas_agrupadas)
df_formatado["Extras"] = df_formatado["Extras"].apply(lambda x: " | ".join(x) if x else "")

In [30]:
# coleta de saldos dia | dados base
df_saldo_dia = df_formatado[df_formatado['SaldoDia'] != ''].copy()
df_saldo_dia['Historico'] = "SALDO DO DIA"
df_saldo_dia_form = df_saldo_dia.drop(columns=['Tipo', 'Valor', 'Extras'])
df_saldo_dia_form.rename(columns={'SaldoDia': 'Valor'}, inplace=True)

In [31]:
# coleta de saldos bloqueado e anterior | dados base
df_saldo_info = df_formatado.drop(columns='SaldoDia')
df_saldo_info_filtrado = df_saldo_info[df_saldo_info['Historico'].str.startswith('SALDO', na=False)]
df_saldo_info_form = df_saldo_info_filtrado.drop(columns=['Tipo', 'Extras'])

In [32]:
# mesclagem de todos os saldos | df_saldo_dia <> df_saldo_info
df_saldo = pd.concat([df_saldo_dia_form, df_saldo_info_form], ignore_index=True)
df_saldo_final = df_saldo.sort_values(by='Data', ascending=True)
df_saldo_final.to_csv("output/saldo_final.csv", index=False)

In [33]:
# tabela base
df_tabela = df_formatado[~df_formatado['Historico'].str.startswith('SALDO', na=False)]
df_tabela_remove_colunas = df_tabela.drop(columns="SaldoDia")


In [34]:
df_tabela_remove_colunas.to_csv("output/extrato.csv", index=False)